In [ ]:
import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('seaborn-v0_8')

In [ ]:
def read_results():
    results = {
        'book_crossing': [],
        'movies': [],
        'food_recipes': [],
        'steam_video_games': []
    }
    for filename in os.listdir('results'):
        for ds in results.keys():
            if ds in filename:
                file_path = os.path.join('results', filename)
                with open(file_path, 'r', encoding='utf-8') as file:
                    result = json.load(file)
                    results[ds].append(result)
    return results

In [ ]:
RESULTS = read_results()
results_list = RESULTS["book_crossing"]

In [ ]:
def plot_solution_1_params(results_list):
    for result in results_list:
        if result['model_name'] == 'LightGCN+ (solution 1)':
            break
    model_params = np.array(result['model_params'])
    user_alpha, item_alpha = model_params[:, 0], model_params[:, 1]
    fig, ax = plt.subplots(figsize=(8, 4))
    if np.unique(user_alpha).shape[0] > 1:
        ax.plot(user_alpha, label=r'$\beta_{user}$')
    if np.unique(item_alpha).shape[0] > 1:
        ax.plot(item_alpha, label=r'$\beta_{item}$')
    ax.set_title(f'{result["model_name"]} parameters', fontweight='bold')
    ax.set_xlabel('epoch', fontweight='bold')
    ax.set_ylabel('value', fontweight='bold')
    ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_solution_1_params(results_list)

In [ ]:
def plot_solution_2_params(results_list):
    for result in results_list:
        if result['model_name'] == 'LightGCN+ (solution 2)':
            break
    model_params = np.array(result['model_params'])
    user_coefs_vector = model_params[:, 0, :]
    item_coefs_vector = model_params[:, 1, :]
    
    fig, axs = plt.subplots(2, 1, figsize=(8, 6))
    if np.unique(item_coefs_vector).shape[0] > 1:
        im_items = axs[0].imshow(item_coefs_vector.T, cmap='viridis', aspect='auto')
        fig.colorbar(im_items, ax=axs[0], fraction=0.046, pad=0.04)
        axs[0].grid(False)

    if np.unique(user_coefs_vector).shape[0] > 1:
        im_users = axs[1].imshow(user_coefs_vector.T, cmap='viridis', aspect='auto')
        fig.colorbar(im_users, ax=axs[1], fraction=0.046, pad=0.04)
        axs[1].grid(False)

    axs[0].set_title('Item Coefficients Vector', fontweight='bold')
    axs[1].set_title('User Coefficients Vector', fontweight='bold')
    axs[0].set_xlabel('epoch', fontweight='bold')
    axs[1].set_xlabel('epoch', fontweight='bold')
    axs[0].set_ylabel('embdedding dimension', fontweight='bold')
    axs[1].set_ylabel('embdedding dimension', fontweight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_solution_2_params(results_list)

In [ ]:
def plot_solution_3_params(results_list, K):
    for result in results_list:
        if result['model_name'] == 'LightGCN+ (solution 3)':
            break

    inds = np.linspace(0, 39, K).astype(int)
    fig, ax = plt.subplots(2, K, figsize=(14, 6))

    for i in range(K):
        user_proj_matrix = np.array(result['model_params'][inds[i]][0])
        item_proj_matrix = np.array(result['model_params'][inds[i]][1])
        im = ax[0, i].imshow(user_proj_matrix, cmap='viridis', aspect='auto')
        fig.colorbar(im, ax=ax[0, i], fraction=0.046, pad=0.04)
        ax[0, i].grid(False)
        im = ax[1, i].imshow(item_proj_matrix, cmap='viridis', aspect='auto')
        fig.colorbar(im, ax=ax[1, i], fraction=0.046, pad=0.04)
        ax[1, i].grid(False)
        ax[1, i].set_title(f'Epoch {inds[i] + 1}', fontweight='bold', fontsize=12)


    ax[0, 0].set_ylabel('User Transformation Matrix', fontweight='bold', fontsize=12)
    ax[1, 0].set_ylabel('Item Transformation Matrix', fontweight='bold', fontsize=12)

    plt.tight_layout()

In [ ]:
plot_solution_3_params(results_list, 5)

In [ ]:
def plot_validation_metrics_for_dataset(dataset_name):
    fig, ax = plt.subplots(2, 1, figsize=(10, 8))
    results_list = RESULTS[dataset_name]
    for result in results_list:
        label = result['model_name']
        ax[0].plot(result['val_recall'], label=label)
        ax[1].plot(result['val_ndcg'], label=label)
        ax[0].set_title('Validation Recall@20', fontweight='bold')
        ax[1].set_title('Validation NDCG@20', fontweight='bold')
        for i in range(2):
            ax[i].set_xlabel('epoch', fontweight='bold')
            ax[i].set_ylabel('value', fontweight='bold')
            ax[i].legend()
    plt.tight_layout()


In [ ]:
plot_validation_metrics_for_dataset('movies')

In [ ]:
RESULTS = read_results()
results_list = RESULTS["food_recipes"]

In [ ]:
training_time = {
    'LightGCN': 0,
    'LightGCN+ (solution 0)': 0,
    'LightGCN+ (solution 1)': 0,
    'LightGCN+ (solution 2)': 0,
    'LightGCN+ (solution 3)': 0
}

model_names = []

for result in results_list:
    n_epochs = len(result['train_loss'])
    training_time[result['model_name']] = result['training_time'] / n_epochs

fig, ax = plt.subplots(figsize=(9, 4))
ax.bar(training_time.keys(), training_time.values(), width=0.25)
ax.set_title('Training time per epoch', fontweight='bold')
ax.set_ylabel('seconds', fontweight='bold')
# ax.set_ylim(bottom=0.95*training_time['LightGCN'])
ax.set_xticklabels(list(training_time.keys()), fontweight='bold')
plt.tight_layout()

In [ ]:
results_summary = []

datasets_names_list = ['book_crossing', 'movies', 'food_recipes']
model_names_list = ['LightGCN', 'LightGCN+ (solution 0)', 'LightGCN+ (solution 1)', 'LightGCN+ (solution 2)', 'LightGCN+ (solution 3)']

for dataset_name in datasets_names_list:
    for model_name in model_names_list:
        results_list = RESULTS[dataset_name]
        for result in results_list:
            if result['model_name'] == model_name:
                best_ind = np.argmax(result['val_ndcg'])
                results_summary.append((dataset_name, model_name, result['val_ndcg'][best_ind], result['val_recall'][best_ind], result['training_time']/40))

results_summary = pd.DataFrame(results_summary, columns=['dataset', 'model_name', 'ndcg', 'recall', 'training_time'])

In [ ]:
results_summary